In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,430 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-02 23:26:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-02 23:26:04 (6.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

# **Filters**

In [ ]:
#1 filter data and create new dataframe to retrieve rows where total_votes count is equal to or greater than 20
total_votes_filter_df = df.filter(df.total_votes>=20)
total_votes_filter_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [ ]:
#2 filter again to create new dataframe to retrieve rows where number of helpful_vote divided by total_votes is equal to or greater than 50%
helpful_votes_filter_df = total_votes_filter_df.filter((df.helpful_votes)/(df.total_votes)*100>=50)
helpful_votes_filter_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [ ]:
#3 filter again to create another dataframe from #2 where reviews are written as part of the vine program(paid), where vine == 'Y'
vine_filter_df = helpful_votes_filter_df.filter(df.vine=='Y')
vine_filter_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36356690|R184FOUNZZ7KO8|B0117RFX38|     294335899|Bose SoundTrue Ar...|     Electronics|          5|           15|         20|   Y|                N|Transparent - Mus...|When I see the Bo...| 2015-08-31|
|         US|   37291590| R82QWN2X2OCHB|B0117RGG8E|     871919611|Bose SoundLink ar...|     Electronics|          5|    

In [ ]:
#4 filter again to create another dataframe from #2 where reviews are NOT written as part of the vine program(unpaid), where vine == 'N'
not_vine_filter_df = helpful_votes_filter_df.filter(df.vine=='N')
not_vine_filter_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [ ]:
#5a total number of paid reviews with total_votes equal to or greater than 20 with 'helpful' votes > 50%
total_paid_count = vine_filter_df.count()
total_paid_count

1080

In [ ]:
#5b total number of paid 5-star reviews
total_paid_5_star = vine_filter_df.filter(vine_filter_df.star_rating == '5').count()
total_paid_5_star

454

In [ ]:
#5c percentage of paid 5-star reviews
paid_5_star_percentage = total_paid_5_star/total_paid_count*100
round(paid_5_star_percentage,2)

42.04

In [ ]:
#5a total number of unpaid reviews with total_votes equal to or greater than 20
#5a total number of paid reviews with total_votes equal to or greater than 20 with 'helpful' votes > 50%
total_unpaid_count = not_vine_filter_df.count()
total_unpaid_count

49673

In [ ]:
#5d total number of unpaid 5-star reviews
total_unpaid_5_star = not_vine_filter_df.filter(not_vine_filter_df.star_rating == '5').count()
total_unpaid_5_star

23043

In [ ]:
#5e percentage of unpaid 5-star reviews
unpaid_5_star_percentage = total_unpaid_5_star/total_unpaid_count*100
round(unpaid_5_star_percentage,2)

46.39